In [1]:
pip install tensorflow keras numpy matplotlib pandas scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow import keras

2024-03-01 19:19:56.810290: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 19:19:56.810337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 19:19:56.811662: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-01 19:19:56.819067: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 19:19:57.659436: W tensorflow/compiler/tf2

In [3]:
import numpy as np
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.layers import *
import pandas as pd
from keras.models import load_model
from keras.preprocessing import image

### Defining the train images path and partial test images path in which we have to add the image to convert it into full path.


In [4]:
train_path = 'DataSet/Train'
test_path = 'DataSet\Test\\'

In [5]:
train_batches = ImageDataGenerator().flow_from_directory(train_path,target_size = (224,224),classes=['RG_Frente','RG_Verso', 'CPF_Frente', 'CPF_Verso', 'CNH_Frente', 'CNH_Verso'],batch_size = 7)

Found 150 images belonging to 6 classes.


### Importing keras pretrained model (without top layers) resnet50 which we are going to use as base model.


In [6]:
base_model = keras.applications.resnet50.ResNet50( weights = 'imagenet',include_top = False,input_shape = (224,224,3))

2024-03-01 19:20:13.936095: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18455 MB memory:  -> device: 0, name: NVIDIA RTX A4500, pci bus id: 0000:c1:00.0, compute capability: 8.6


In [7]:
for layer in base_model.layers:
    layer.trainable = False

### Making keras sequential model by using base model layer as its lower layer and two fully connected dense layer as its top layer which are trainable.

In [15]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
# model.add(Dense(1000,activation = 'relu'))
model.add(Dense(6,activation = 'sigmoid'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 100352)            0         
                                                                 
 dense_4 (Dense)             (None, 6)                 602118    
                                                                 
Total params: 24189830 (92.28 MB)
Trainable params: 602118 (2.30 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________
None


### Compile and train the model on epoch = 5

In [16]:
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [17]:
history = model.fit(train_batches,
                              epochs=20,
                              steps_per_epoch = 7,
                             )

Epoch 1/20
7/7 [==============================] - 3s 49ms/step - loss: 1.8260 - accuracy: 0.5306
Epoch 2/20
7/7 [==============================] - 0s 68ms/step - loss: 0.3643 - accuracy: 0.9184
Epoch 3/20
7/7 [==============================] - 0s 61ms/step - loss: 0.0798 - accuracy: 1.0000
Epoch 4/20
7/7 [==============================] - 0s 65ms/step - loss: 0.0261 - accuracy: 1.0000
Epoch 5/20
7/7 [==============================] - 0s 61ms/step - loss: 0.1777 - accuracy: 0.9388
Epoch 6/20
7/7 [==============================] - 0s 63ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 7/20
7/7 [==============================] - 0s 63ms/step - loss: 0.0371 - accuracy: 1.0000
Epoch 8/20
7/7 [==============================] - 0s 66ms/step - loss: 7.2064e-06 - accuracy: 1.0000
Epoch 9/20
7/7 [==============================] - 1s 70ms/step - loss: 0.0517 - accuracy: 1.0000
Epoch 10/20
7/7 [==============================] - 1s 70ms/step - loss: 4.6703e-07 - accuracy: 1.0000
Epoch 11/20
7/7 [====

In [18]:
model.save('lunar-v7.h5')
print("lunar keras saved")

lunar keras saved


In [44]:
def predict(img_path):
    test_image = image.load_img(img_path, target_size=(224, 224))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = model.predict(test_image, batch_size=1)

    print(type(result)) 
    print(result.shape)

    return result
    # # # Softmax
    # # # Apply Softmax 
    # probabilities = softmax(result[0])

    # # # Find the most likely class
    # predicted_class = np.argmax(probabilities)
      
    # # # Mapping classes to labels (add your labels here)
    # class_labels = ["RG Frente", "RG Verso", "CPF Frente", "CPF Verso", "CNH Frente", "CNH Verso"]
    
    # # # Print results  
    # print(f"Predicted Class: {class_labels[predicted_class]}, Probabilities: {probabilities}") 
    
    # # # Remove the hardcoded if/else logic 
    # return class_labels[predicted_class] 

label = predict('DataSet/Predict/rf1.jpg')
print(label[0])

1/1 [==============================] - 0s 34ms/step
<class 'numpy.ndarray'>
(1, 6)
[-1063.0544   -988.35547  -542.79407  -117.1619   -564.07336  -314.77563]


In [41]:
softmax_layers = keras.layers.activations.Softmax()

input = np.array([1.0, 2.0, 1.0])

result = softmax_layers(input)

print(result)

AttributeError: module 'keras.api._v2.keras.layers' has no attribute 'activations'

In [13]:
import os

def get_and_predict_jpgs(directory_path):
  for filename in os.listdir(directory_path):
      if filename.endswith(".jpg") or filename.endswith(".jpeg"):
          img_path = os.path.join(directory_path, filename)
          prediction = predict(img_path)
          print(f"Image: {filename} - Prediction: {prediction}")

In [35]:
label = predict('DataSet/Predict/rf1.jpg')
print(label)

1/1 [==============================] - 0s 33ms/step
<class 'numpy.ndarray'>
(1, 6)


AttributeError: 'tuple' object has no attribute 'rank'

In [26]:
label = predict('DataSet/Predict/rv1.jpg')
print(label)

1/1 [==============================] - 0s 34ms/step
[[3.7555170e-32 1.0000000e+00 4.8909164e-37 0.0000000e+00 2.3450348e-27
  0.0000000e+00]]
[3.7555170e-32 1.0000000e+00 4.8909164e-37 0.0000000e+00 2.3450348e-27
 0.0000000e+00]
1.0
RG Verso


In [27]:
directory_path = 'DataSet/Predict'
get_and_predict_jpgs(directory_path) 

1/1 [==============================] - 0s 37ms/step
[[0. 0. 0. 1. 0. 0.]]
[0. 0. 0. 1. 0. 0.]
0.0
Image: cpf_verso1.jpg - Prediction: CPF Verso
1/1 [==============================] - 0s 26ms/step
[[0.0000000e+00 0.0000000e+00 2.4562238e-21 1.0000000e+00 0.0000000e+00
  0.0000000e+00]]
[0.0000000e+00 0.0000000e+00 2.4562238e-21 1.0000000e+00 0.0000000e+00
 0.0000000e+00]
0.0
Image: cpf_verso0.jpg - Prediction: CPF Verso
1/1 [==============================] - 0s 21ms/step
[[0. 0. 0. 1. 0. 0.]]
[0. 0. 0. 1. 0. 0.]
0.0
Image: cpf_verso5.jpg - Prediction: CPF Verso
1/1 [==============================] - 0s 23ms/step
[[0. 0. 0. 1. 0. 0.]]
[0. 0. 0. 1. 0. 0.]
0.0
Image: cpf_verso7.jpg - Prediction: CPF Verso
1/1 [==============================] - 0s 24ms/step
[[0. 0. 0. 1. 0. 0.]]
[0. 0. 0. 1. 0. 0.]
0.0
Image: cpf_verso6.jpg - Prediction: CPF Verso
1/1 [==============================] - 0s 22ms/step
[[0. 0. 0. 1. 0. 0.]]
[0. 0. 0. 1. 0. 0.]
0.0
Image: cpf_verso4.jpg - Prediction: CPF Verso
1